In [3]:
print("Hello World")

Hello World


### Importing Libraries

In [1]:
import pandas as pd

In [2]:
#libraries to connect to postgresssql
from dotenv import load_dotenv
import os
import psycopg2
from sqlalchemy import create_engine
load_dotenv()


True

## Loading Data Set and Performing Data Cleaning

In [3]:
df = pd.read_csv('Walmart.csv',encoding_errors='ignore')

In [4]:
df.shape

(10051, 11)

In [5]:
df.head()


,invoice_id,Branch,City,category,unit_price,quantity,date,time,payment_method,rating,profit_margin
0,1,WALM003,San Antonio,Health and beauty,$74.69,7.0,05/01/19,13:08:00,Ewallet,9.1,0.48
1,2,WALM048,Harlingen,Electronic accessories,$15.28,5.0,08/03/19,10:29:00,Cash,9.6,0.48
2,3,WALM067,Haltom City,Home and lifestyle,$46.33,7.0,03/03/19,13:23:00,Credit card,7.4,0.33
3,4,WALM064,Bedford,Health and beauty,$58.22,8.0,27/01/19,20:33:00,Ewallet,8.4,0.33
4,5,WALM013,Irving,Sports and travel,$86.31,7.0,08/02/19,10:37:00,Ewallet,5.3,0.48


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10051 entries, 0 to 10050
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   invoice_id      10051 non-null  int64  
 1   Branch          10051 non-null  object 
 2   City            10051 non-null  object 
 3   category        10051 non-null  object 
 4   unit_price      10020 non-null  object 
 5   quantity        10020 non-null  float64
 6   date            10051 non-null  object 
 7   time            10051 non-null  object 
 8   payment_method  10051 non-null  object 
 9   rating          10051 non-null  float64
 10  profit_margin   10051 non-null  float64
dtypes: float64(3), int64(1), object(7)
memory usage: 863.9+ KB


In [7]:
# Finding total number of duplicate value
df.duplicated().sum()

np.int64(51)

In [10]:
#Removing Duplicate
df.drop_duplicates(inplace=True)
df.duplicated().sum()

np.int64(0)

In [11]:
df.isnull().sum()

invoice_id         0
Branch             0
City               0
category           0
unit_price        31
quantity          31
date               0
time               0
payment_method     0
rating             0
profit_margin      0
dtype: int64

In [12]:
# dropping rows were column is null
df.dropna(inplace=True)

In [13]:
df.isnull().sum()

invoice_id        0
Branch            0
City              0
category          0
unit_price        0
quantity          0
date              0
time              0
payment_method    0
rating            0
profit_margin     0
dtype: int64

In [14]:
# Checking Data Types
df.dtypes

invoice_id          int64
Branch             object
City               object
category           object
unit_price         object
quantity          float64
date               object
time               object
payment_method     object
rating            float64
profit_margin     float64
dtype: object

In [15]:
df['unit_price']=df['unit_price'].str.replace('$','').astype('float')
df.head()

,invoice_id,Branch,City,category,unit_price,quantity,date,time,payment_method,rating,profit_margin
0,1,WALM003,San Antonio,Health and beauty,74.69,7.0,05/01/19,13:08:00,Ewallet,9.1,0.48
1,2,WALM048,Harlingen,Electronic accessories,15.28,5.0,08/03/19,10:29:00,Cash,9.6,0.48
2,3,WALM067,Haltom City,Home and lifestyle,46.33,7.0,03/03/19,13:23:00,Credit card,7.4,0.33
3,4,WALM064,Bedford,Health and beauty,58.22,8.0,27/01/19,20:33:00,Ewallet,8.4,0.33
4,5,WALM013,Irving,Sports and travel,86.31,7.0,08/02/19,10:37:00,Ewallet,5.3,0.48


In [26]:
# Adding new coloum total

In [16]:
df['total']= df['unit_price']*df['quantity']
df.head()

,invoice_id,Branch,City,category,unit_price,quantity,date,time,payment_method,rating,profit_margin,total
0,1,WALM003,San Antonio,Health and beauty,74.69,7.0,05/01/19,13:08:00,Ewallet,9.1,0.48,522.83
1,2,WALM048,Harlingen,Electronic accessories,15.28,5.0,08/03/19,10:29:00,Cash,9.6,0.48,76.40
2,3,WALM067,Haltom City,Home and lifestyle,46.33,7.0,03/03/19,13:23:00,Credit card,7.4,0.33,324.31
3,4,WALM064,Bedford,Health and beauty,58.22,8.0,27/01/19,20:33:00,Ewallet,8.4,0.33,465.76
4,5,WALM013,Irving,Sports and travel,86.31,7.0,08/02/19,10:37:00,Ewallet,5.3,0.48,604.17


In [17]:
df.to_csv('walmart_clean_data.csv',index=False)

## Connecting to Postgress SQL

In [23]:

def fetch_data_from_postgres():
    try:
        print("Entered Try")
      
        
        conn = psycopg2.connect(
            dbname="walmart_db",
            user=os.getenv("PG_USER"),
            password=os.getenv("PG_PASSWORD"),
            host=os.getenv("PG_HOST"),
            port=os.getenv("PG_PORT")
        )

        cur = conn.cursor()
        print("Connection established")
    except Exception as e:
        print("Error:", e)

fetch_data_from_postgres()

Entered Try
Connection established


In [18]:
def import_csv_to_postgres():
    try:
       
        
        required_vars = ["PG_USER", "PG_PASSWORD", "PG_HOST", "PG_PORT", "PG_DB"]
        for var in required_vars:
            if os.getenv(var) is None:
                raise ValueError(f"Environment variable {var} is missing or not set.")

        # Create SQLAlchemy engine (PostgreSQL connection)
        engine = create_engine(f"postgresql+psycopg2://{os.getenv('PG_USER')}:{os.getenv('PG_PASSWORD')}@{os.getenv('PG_HOST')}:{os.getenv('PG_PORT')}/{os.getenv('PG_DB')}")
        print("After engine")

        # Export DataFrame to PostgreSQL (creates table if not exists)
        df.to_sql(name='walmart', con=engine, if_exists='append', index=False)

        print("Successfull")

    except Exception as e:
        
        print("Error:", e)

import_csv_to_postgres()

After engine
Error: (psycopg2.errors.InsufficientPrivilege) permission denied for schema public
LINE 2: CREATE TABLE walmart (
                     ^

[SQL: 
CREATE TABLE walmart (
	invoice_id BIGINT, 
	"Branch" TEXT, 
	"City" TEXT, 
	category TEXT, 
	unit_price FLOAT(53), 
	quantity FLOAT(53), 
	date TEXT, 
	time TEXT, 
	payment_method TEXT, 
	rating FLOAT(53), 
	profit_margin FLOAT(53), 
	total FLOAT(53)
)

]
(Background on this error at: https://sqlalche.me/e/20/f405)
